In [1]:
import numpy as np
from enhanced_hybrid_hhl import (HHL, 
                                 Lee_preprocessing,  
                                 HybridInversion, 
                                 QuantumLinearSystemProblem, 
                                 QuantumLinearSystemSolver,
                                 EnhancedHybridInversion)

In [2]:
from qiskit_aer import AerSimulator

# define the backend to run the circuits on
simulator = AerSimulator()

In [3]:
# Define quantum linear system problem to be solved with HHL
A_matrix = np.array([[ 0.5 , -0.25],
        [-0.25,  0.5 ]])
b_vector = np.array([[1.], [0.]])
problem = QuantumLinearSystemProblem(A_matrix=A_matrix,
                                     b_vector=b_vector)

# Define Parameters
k = 3 # clock qubits for hhl.
l = k+2 # clock qubits for enhanced preprocessing.
min_prob = 2**-k # hybrid preprocessing relevance threshold.
relevance_threshold = 2**-l # enhanced hybrid preprocessing relevance threshold.
maximum_eigenvalue = 1 # Over estimate of largest eigenvalue in the system.

get_result_type = 'get_swap_test_result'
ideal_x_statevector = QuantumLinearSystemSolver(problem=problem).ideal_x_statevector

In [4]:
# Define a preprocessing function. In this example, we use the standard QPEA used by Lee et al.
enhanced_preprocessing = Lee_preprocessing(num_eval_qubits=l,
                                  max_eigenvalue= maximum_eigenvalue, 
                                  backend=simulator).estimate

enhanced_eigenvalue_inversion = EnhancedHybridInversion

enhanced_hybrid_hhl = HHL(get_result_function= get_result_type,
          preprocessing= enhanced_preprocessing,
          eigenvalue_inversion= enhanced_eigenvalue_inversion,
          backend=simulator,
          statevector=ideal_x_statevector)

enhanced_hybrid_hhl_result = enhanced_hybrid_hhl.estimate(problem=problem,
                                                          num_clock_qubits=k,
                                                          max_eigenvalue=1)

print(enhanced_hybrid_hhl_result)

{   'circuit_results': <qiskit_aer.jobs.aerjob.AerJob object at 0x00000258FEC60A60>,
    'control_state_list': None,
    'eigenbasis_projection_list': [0.051, 0.048, 0.3965, 0.40825, 0],
    'eigenvalue_list': [0.2, 0.8, 0.7333333333333333, 0.26666666666666666, 1],
    'ideal_x_statevector': None,
    'post_processing': None,
    'results_processed': 0.9664474215169189,
    'rotation_angle_list': None,
    'shots': None}
